In [1]:
import math
import pandas as pd
import numpy as np
import joblib

In [2]:
def L(x):
    return math.log10(x / (1 - x))

In [3]:
with open('encoder.pickle', 'rb') as f:
    encoder = joblib.load(f)

In [4]:
# (Score, CSV)
configs = [
    (0.851726254, "efficient_b7_ap_foldall.csv"), # b7_ap_sota_tta
    (0.8586881402, "efficientb8_ap_ms.csv") # b8_ap_sota_tta
]
output = "ensemble_result.csv"

In [5]:
csv_files = []
score_sum = 0
for score, path in configs:
    score_sum += L(score)
    csv_files.append(pd.read_csv(path))

coeff = [L(score) / score_sum for score, _ in configs]
pred = []
for csv in csv_files:
    pred.append(csv['score'].to_list())
    
pred_total = []
for i in range(len(configs)):
    current_pred = []
    for score in pred[i]:
        current_pred.append(list(map(float, score[1:-1].split(','))))
    pred_total.append(current_pred)

print(coeff)

[0.4920859438045291, 0.5079140561954709]


In [6]:
X = np.zeros((len(pred_total[0]), len(pred_total[0][0])))
for idx, current_pred in enumerate(pred_total):
    X += np.array(current_pred) * coeff[idx]
    
y = X.argmax(1)
y = encoder.inverse_transform(y)
pred = y.tolist()

In [7]:
submission = pd.read_csv('test_data.csv')
submission['label'] = pred
submission

,index,file_name,path,label
0,0,20000.png,/dacon/dacon/anomaly-detection/data/test/20000...,tile-glue_strip
1,1,20001.png,/dacon/dacon/anomaly-detection/data/test/20001...,grid-good
2,2,20002.png,/dacon/dacon/anomaly-detection/data/test/20002...,transistor-good
3,3,20003.png,/dacon/dacon/anomaly-detection/data/test/20003...,tile-gray_stroke
4,4,20004.png,/dacon/dacon/anomaly-detection/data/test/20004...,tile-good
...,...,...,...,...
2149,2149,22149.png,/dacon/dacon/anomaly-detection/data/test/22149...,tile-gray_stroke
2150,2150,22150.png,/dacon/dacon/anomaly-detection/data/test/22150...,screw-good
2151,2151,22151.png,/dacon/dacon/anomaly-detection/data/test/22151...,grid-good
2152,2152,22152.png,/dacon/dacon/anomaly-detection/data/test/22152...,cable-good


In [8]:
del submission['path']
del submission['file_name']
submission.to_csv(output, index=None)